In [1]:
import talib
import numpy as np
import pandas as pd
import random
from collections import deque

In [2]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2020-2021.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

df_small = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2020-2021.csv')
df_small.Date = pd.to_datetime(df_small.Date)
df_small.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2020-07-30 07:00:00   1.29543   1.29607  1.29540    1.29572   1.29562   
1 2020-07-30 07:05:00   1.29569   1.29591  1.29505    1.29542   1.29588   
2 2020-07-30 07:10:00   1.29541   1.29590  1.29514    1.29559   1.29557   
3 2020-07-30 07:15:00   1.29561   1.29567  1.29510    1.29515   1.29577   
4 2020-07-30 07:20:00   1.29514   1.29643  1.29501    1.29642   1.29529   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.29623  1.29560    1.29589   1.29552   1.29615  1.29550    1.29580  
1   1.29608  1.29522    1.29560   1.29578   1.29600  1.29514    1.29551  
2   1.29606  1.29535    1.29575   1.29549   1.29598  1.29524    1.29567  
3   1.29584  1.29530    1.29533   1.29569   1.29576  1.29520    1.29524  
4   1.29660  1.29518    1.29660   1.29522   1.29652  1.29510    1.29651  
---------------------------------------------------------------------------
                     Date  Bi

In [5]:
print(df_small.head())
print('-' * 75)
print(df_small.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2020-07-30 07:00:00   1.29543   1.29607  1.29540    1.29572   1.29562   
1 2020-07-30 07:05:00   1.29569   1.29591  1.29505    1.29542   1.29588   
2 2020-07-30 07:10:00   1.29541   1.29590  1.29514    1.29559   1.29557   
3 2020-07-30 07:15:00   1.29561   1.29567  1.29510    1.29515   1.29577   
4 2020-07-30 07:20:00   1.29514   1.29643  1.29501    1.29642   1.29529   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.29623  1.29560    1.29589   1.29552   1.29615  1.29550    1.29580  
1   1.29608  1.29522    1.29560   1.29578   1.29600  1.29514    1.29551  
2   1.29606  1.29535    1.29575   1.29549   1.29598  1.29524    1.29567  
3   1.29584  1.29530    1.29533   1.29569   1.29576  1.29520    1.29524  
4   1.29660  1.29518    1.29660   1.29522   1.29652  1.29510    1.29651  
---------------------------------------------------------------------------
                     Date  Bi

In [6]:
def add_fractal(df, i, look_back=2):
  if i >= look_back and i < df.shape[0] - look_back:
    lows = []
    highs = []

    for j in range(1, look_back + 1):
      prev_bid_low, prev_bid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
      future_bid_low, future_bid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

      lows.append(float(prev_bid_low))
      lows.append(float(future_bid_low))
      highs.append(float(prev_bid_high))
      highs.append(float(future_bid_high))

    bid_low, bid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

    if float(bid_low) < min(lows):
      return 1

    elif float(bid_high) > max(highs):
      return 2

    else:
      return 0

  else:
    return np.nan

In [7]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['ema50'] = talib.EMA(df['Mid_Close'], timeperiod=50)
df['ema30'] = talib.EMA(df['Mid_Close'], timeperiod=30)
df['ema25'] = talib.EMA(df['Mid_Close'], timeperiod=25)
df['ema20'] = talib.EMA(df['Mid_Close'], timeperiod=20)
df['ema15'] = talib.EMA(df['Mid_Close'], timeperiod=15)
df['atr'] = talib.ATR(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=500)
df['adx'] = talib.ADX(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=14)
df['slowk'], df['slowd'] = talib.STOCH(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['fractal'] = [add_fractal(df, i) for i in range(df.shape[0])]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# print(df['fractal'].value_counts())

In [8]:
value_per_pip = 1.0
max_pips_to_risk = 0.0100
amounts_per_day = [-4, -5, -6]

In [9]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_simulation(risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff, max_open_trades=1):
    pullback_cushion /= 10000
    fractal_distance /= 10000
    bar_length /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000 / max_open_trades
    trades = deque(maxlen=max_open_trades)
    time_window = 10
    stoch_signal = None
    z = 0

    def iterate_thru_small_data(trade):
        def get_end_date(j):
          return df_small.loc[df_small.index[j + 1], 'Date'] if j + 1 < len(df_small) else None

        j = df_small[df_small.Date >= trade['start_date']].index[0]

        nonlocal reward
        nonlocal day_fees
        nonlocal n_losses
        nonlocal n_wins
        nonlocal curr_win_streak
        nonlocal curr_loss_streak
        nonlocal win_streak
        nonlocal loss_streak

        while j < len(df_small):
          curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_date = \
          df_small.loc[df_small.index[j], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low',
                                         'Ask_Close', 'Date']]

          if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
              trade['stop_loss'] = curr_bid_high - trade['pips_risked']

          if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
            trade['stop_loss'] = trade['pips_risked'] + curr_ask_low

          if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          j += 1


    i = 2

    while i < len(df):
        if stoch_signal is not None:
          z += 1

        if z >= time_window:
          stoch_signal = None
          z = 0

        curr_date = df.loc[df.index[i], 'Date']

        # Remove any expired trades -----------------------------------------
        trades_to_remove = []

        for trade in trades:
          if trade['end_date'] is not None and trade['end_date'] >= curr_date:
            trades_to_remove.append(trade)

        for trade in trades_to_remove:
          trades.remove(trade)
        # -------------------------------------------------------------------

        ema200 = df.loc[df.index[i - 1], 'ema200']
        adx = df.loc[df.index[i - 1], 'adx']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        enough_volatility = spread <= spread_cutoff
        curr_al = df.loc[df.index[i - 1], 'Ask_Low']
        curr_bh = df.loc[df.index[i - 1], 'Bid_High']
        curr_ml = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh - curr_ml)
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
        slowk1, slowd1 = df.loc[df.index[i - 1], ['slowk', 'slowd']]
        slowk2, slowd2 = df.loc[df.index[i - 2], ['slowk', 'slowd']]
        trending = adx >= adx_cutoff

        if slowk2 < slowd2 and slowk1 > slowd1 and max([20, slowk2, slowd2, slowk1, slowd1]) == 20:
          stoch_signal = 'buy'
          z = 0

        elif slowk2 > slowd2 and slowk1 < slowd1 and min([80, slowk2, slowd2, slowk1, slowd1]) == 80:
          stoch_signal = 'sell'
          z = 0

        process_trade = False

        if macd2 < macdsignal2 and macd1 > macdsignal1 and stoch_signal == 'buy' and curr_ml > ema200 and enough_volatility and trending and len(trades) < max_open_trades and curr_bar_length <= bar_length:
            all_buys = True

            for trade in trades:
                if trade['trade_type'] != 'buy':
                    all_buys = False
                    break

            if all_buys:
              pullback = None
              u = i - 3

              while u >= 0:
                  if df.loc[df.index[u], 'fractal'] == 1:
                      curr_pullback = float(df.loc[df.index[u], 'Bid_Low'])
                      curr_fractal_distance = curr_ao - curr_pullback

                      if curr_fractal_distance >= fractal_distance:
                          pullback = curr_pullback
                          break

                  u -= 1

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_ask_open)
                  stop_loss = round(pullback - pullback_cushion, 5)
                  # stop_loss = round(pullback - pullback_cushion, 3)

                  if stop_loss < open_price:
                      curr_pips_to_risk = open_price - stop_loss

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                            'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                            'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_buys += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        elif macd2 > macdsignal2 and macd1 < macdsignal1 and stoch_signal == 'sell' and curr_mh < ema200 and enough_volatility and trending and len(trades) < max_open_trades and curr_bar_length <= bar_length:
            all_sells = True

            for trade in trades:
                if trade['trade_type'] != 'sell':
                    all_sells = False
                    break

            if all_sells:
              pullback = None
              u = i - 3

              while u >= 0:
                  if df.loc[df.index[u], 'fractal'] == 2:
                      curr_pullback = float(df.loc[df.index[u], 'Ask_High'])
                      curr_fractal_distance = curr_pullback - curr_bo

                      if curr_fractal_distance >= fractal_distance:
                          pullback = curr_pullback
                          break

                  u -= 1

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_bid_open)
                  stop_loss = round(pullback + pullback_cushion, 5)
                  # stop_loss = round(pullback + pullback_cushion, 3)

                  if stop_loss > open_price:
                      curr_pips_to_risk = stop_loss - open_price

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                  'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                  'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_sells += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        if process_trade:
          iterate_thru_small_data(trades[-1])

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak

In [10]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.8, 1.9, 2]
# risk_reward_ratio_vals = [2]
pullback_cushion_vals = [5, 10, 15]
# pullback_cushion_vals = [15]
fractal_distances = [5, 10, 15, 20]
# fractal_distances = [20]
bar_lengths = [5, 10]
# bar_lengths = [5]
adx_cutoffs = [20, 25, 30]
# adx_cutoffs = [0]
spread_cutoffs = [2, 3, 4]
# spread_cutoffs = [4]
n_possibilities = len(risk_reward_ratio_vals) * len(pullback_cushion_vals) * len(fractal_distances) * len(bar_lengths) * len(adx_cutoffs) * len(spread_cutoffs)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
  for pullback_cushion in pullback_cushion_vals:
    for fractal_distance in fractal_distances:
      for bar_length in bar_lengths:
        for adx_cutoff in adx_cutoffs:
          for spread_cutoff in spread_cutoffs:
            all_combos.append((risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_pullback_cushion = None
best_last_x_bars_val = None
best_fractal_distance = None
best_bar_length = None
best_adx_cutoff = None
best_spread_cutoff = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff in combos_to_try:
  reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak = run_simulation(risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff)
  runs_finished += 1

  print(reward)
  print('Num buys: ' + str(n_sells))
  print('Num sells: ' + str(n_buys))
  print('Num trades: ' + str(n_buys + n_sells))
  print('Num wins: ' + str(n_wins))
  print('Num losses: ' + str(n_losses))
  print('Win streak: ' + str(win_streak))
  print('Loss streak: ' + str(loss_streak))
  print('Remaining runs: ' + str(n_runs - runs_finished))

  min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None
  if min_item is None or reward > min_item['reward']:
    if min_item is not None:
      best_rewards.remove(min_item)
    best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'pullback': pullback_cushion, 'fractal_dist': fractal_distance, 'bar_length': bar_length, 'adx': adx_cutoff, 'spread': spread_cutoff})


  if reward > best_reward:
    best_reward = reward
    best_risk_reward = risk_reward_ratio
    best_pullback_cushion = pullback_cushion
    best_fractal_distance = fractal_distance
    best_bar_length = bar_length
    best_adx_cutoff = adx_cutoff
    best_spread_cutoff = spread_cutoff

  print('Best reward so far: ' + str(best_reward))
  print()

Num runs: 648

-223.49999999999264
Num buys: 21
Num sells: 25
Num trades: 46
Num wins: 15
Num losses: 30
Win streak: 3
Loss streak: 6
Remaining runs: 647
Best reward so far: -223.49999999999264

-1844.5000000000744
Num buys: 79
Num sells: 96
Num trades: 175
Num wins: 59
Num losses: 115
Win streak: 4
Loss streak: 9
Remaining runs: 646
Best reward so far: -223.49999999999264

-1487.4999999999668
Num buys: 27
Num sells: 31
Num trades: 58
Num wins: 17
Num losses: 40
Win streak: 4
Loss streak: 16
Remaining runs: 645
Best reward so far: -223.49999999999264

-1245.4999999999438
Num buys: 47
Num sells: 66
Num trades: 113
Num wins: 43
Num losses: 69
Win streak: 4
Loss streak: 9
Remaining runs: 644
Best reward so far: -223.49999999999264

-2295.9999999999523
Num buys: 48
Num sells: 68
Num trades: 116
Num wins: 34
Num losses: 81
Win streak: 4
Loss streak: 8
Remaining runs: 643
Best reward so far: -223.49999999999264

894.0000000000248
Num buys: 21
Num sells: 25
Num trades: 46
Num wins: 16
Num los

KeyboardInterrupt: 

In [11]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best pullback cushion: ' + str(best_pullback_cushion))
print('Best fractal distance: ' + str(best_fractal_distance))
print('Best bar length: ' + str(best_bar_length))
print('Best spread: ' + str(best_spread_cutoff))
print('Best adx: ' + str(best_adx_cutoff))
print('-----------------------')
print('Top results:')
for entry in best_rewards:
    print(entry)

# 2020-2021:
# {'reward': 1502, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 1504, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1733, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 1504, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1610, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 1666, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1595, 'ratio': 1.8, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 1731, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1503, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 1563, 'ratio': 1.8, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 1728, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 1594, 'ratio': 1.8, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1613, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 1504, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 1598, 'ratio': 1.8, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 1664, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 1562, 'ratio': 1.8, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1663, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 1614, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1561, 'ratio': 1.8, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 4}

------------ FINAL RESULTS ------------
Best reward: 945.5000000000248
Best risk/reward ratio: 1.8
Best pullback cushion: 15
Best fractal distance: 20
Best bar length: 10
Best spread: 3
Best adx: 30
-----------------------
Top results:
{'reward': 894, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 2}
{'reward': 274, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 4}
{'reward': 101, 'ratio': 1.8, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 3}
{'reward': 103, 'ratio': 1.8, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 2}
{'reward': 945, 'ratio': 1.8, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 3}
{'reward': 97, 'ratio': 1.8, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 4}
{'reward': 942, 'ratio': 1.8, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 4}
{'reward': 